In [1]:
import pymc3 as pm
import arviz as az
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import warnings
from theano import shared,tensor
import time 
from datetime import datetime
from sklearn.metrics import mean_squared_error
import pickle

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df=pd.read_csv('/kaggle/input/historyapr/EPA_OD_202304.csv')
df=df[df['PublishTime']>='2023-04-24 00:00:00']
truth=copy.deepcopy(df)
df.head()

,SiteName,County,AQI,Pollutant,Status,SO2,CO,CO_8hr,O3,O3_8hr,...,NO2,NOx,NO,WindSpeed,WindDirec,PublishTime,SO2_AVG,Longitude,Latitude,SiteId
42427,永和,新北市,89.0,懸浮微粒,普通,0.0,0.30,0.3,40.7,43.7,...,10.5,10.8,0.2,1.1,95,2023-04-24 00:00:00,0.0,121.516306,25.017000,70
42428,復興,高雄市,77.0,細懸浮微粒,普通,1.6,0.38,0.3,24.7,43.7,...,18.4,18.5,0.0,0.7,284,2023-04-24 00:00:00,1.0,120.312017,22.608711,71
42429,富貴角,新北市,100.0,懸浮微粒,普通,0.2,0.14,0.1,61.5,64.1,...,1.2,1.2,0.0,4.8,99,2023-04-24 00:00:00,0.0,121.536569,25.296817,84
42430,花蓮,花蓮縣,67.0,細懸浮微粒,普通,0.7,0.27,0.2,44.7,49.2,...,4.7,4.5,0.0,0.3,311,2023-04-24 00:00:00,0.0,121.599769,23.971306,63
42431,宜蘭,宜蘭縣,77.0,懸浮微粒,普通,1.0,0.20,0.2,42.8,50.7,...,2.9,3.4,0.5,0.3,221,2023-04-24 00:00:00,0.0,121.746394,24.747917,65


In [3]:
df.describe()

,AQI,SO2,CO,CO_8hr,O3,O3_8hr,PM10,PM10_AVG,PM2.5,PM2.5_AVG,NO2,NOx,NO,SO2_AVG,Longitude,Latitude,SiteId
count,10000.000000,9767.000000,9838.000000,10001.000000,9788.000000,9929.000000,9901.000000,9997.000000,9858.000000,9955.000000,9749.000000,9749.000000,9749.000000,9992.000000,10010.000000,10010.000000,10010.000000
mean,61.794400,1.427849,0.361061,0.310099,39.149438,38.885779,38.103525,38.804641,18.399371,18.165545,11.365443,14.055790,2.651554,0.877502,120.845289,24.082771,39.610390
std,21.146235,2.783406,0.201773,0.176709,18.765131,15.434078,20.829667,19.932071,9.204587,8.053712,8.125457,13.126819,6.540901,0.776498,0.611859,0.984621,23.123167
min,2.000000,0.000000,0.090000,0.100000,0.000000,0.200000,0.000000,1.000000,0.000000,0.000000,0.100000,0.300000,0.000000,0.000000,118.312256,21.958069,1.000000
25%,46.000000,0.800000,0.250000,0.200000,26.100000,28.900000,24.000000,25.000000,12.000000,13.000000,5.600000,6.400000,0.600000,0.000000,120.348742,23.113376,20.000000
50%,60.000000,1.200000,0.310000,0.300000,39.400000,39.100000,34.000000,35.000000,17.000000,18.000000,9.400000,10.600000,1.000000,1.000000,120.759568,24.162197,39.000000
75%,74.000000,1.800000,0.400000,0.300000,52.100000,49.500000,46.000000,47.000000,23.000000,23.000000,14.900000,16.800000,2.000000,1.000000,121.451861,25.017000,59.000000
max,182.000000,255.400000,2.690000,1.700000,136.300000,98.200000,180.000000,128.000000,65.000000,51.000000,52.900000,155.700000,117.200000,4.000000,121.792928,26.151884,84.000000


In [4]:
features=['AQI','PM10','PM2.5_AVG','PM2.5','SiteId']
for feature in features:
    print(f"{feature} has nan value: ",df[feature].isna().any())
    print(df[df[feature].isna()].index)

AQI has nan value:  True
Int64Index([43627, 43863, 44049, 44555, 44646, 44672, 51655, 51667, 51788,
            51884],
           dtype='int64')
PM10 has nan value:  True
Int64Index([43244, 43290, 43293, 43294, 43366, 43369, 43379, 43381, 43393,
            43418,
            ...
            49749, 49750, 49865, 49869, 50183, 51557, 51655, 51667, 51788,
            52420],
           dtype='int64', length=109)
PM2.5_AVG has nan value:  True
Int64Index([42878, 42902, 43020, 43118, 43126, 43252, 43359, 43473, 43570,
            43627, 43863, 44049, 44555, 44646, 44672, 45139, 45253, 45277,
            45337, 45394, 45650, 47050, 47165, 47262, 47279, 47402, 47499,
            47519, 47548, 47629, 47643, 47665, 47725, 47754, 47777, 47869,
            47874, 47910, 47970, 47976, 47978, 48021, 48099, 48104, 48111,
            48140, 48197, 48216, 48240, 48322, 48347, 51655, 51667, 51788,
            51884],
           dtype='int64')
PM2.5 has nan value:  True
Int64Index([42666, 42785, 42878

In [5]:
df=df[features]
df=df.dropna()

In [6]:
for feature in features[:-2]:
    df[feature] =( df[feature] - df[feature].mean() ) / df[feature].std()

In [7]:
Site=list(df.SiteId.unique())
Site.sort()

In [8]:
len(Site)

77

In [9]:
SiteTrace={}
number=1
for idx,s in enumerate(Site):
    with pm.Model() as model:
        X0=pm.Data('X0',df[df.SiteId==s][features[0]])
        X1=pm.Data('X1',df[df.SiteId==s][features[1]])
        X2=pm.Data('X2',df[df.SiteId==s][features[2]])
        y=pm.Data('y',df[df.SiteId==s][features[3]])
        alpha=pm.Normal('alpha',mu=0,sd=10)
        beta=pm.Normal('beta',mu=0,sd=10,shape=3)
        esp=pm.HalfNormal('esp',sd=10)
        mu=pm.Deterministic('mu',alpha+beta[0]*X0+beta[1]*X1+beta[2]*X2)
        y_pred=pm.Normal('y_pred',mu=mu,sd=esp,observed=y)
        trace=pm.sample()
        ppc=pm.sample_posterior_predictive(trace)
        SiteTrace[s]=trace
        if (idx+1)%10==0 or idx==len(Site)-1:
            pklname="dict"+str(number)+".pkl"
            with open(pklname, 'wb') as file:
                pickle.dump(SiteTrace, file)
            SiteTrace={}
            number+=1
#         SiteTrace[s]={'alpha':trace['alpha'].mean(),'beta0':trace['beta'][0].mean(),'beta1':trace['beta'][1].mean()}

/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


...

/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


In [10]:
ref={}
for i in range(1,9):
    filename="/kaggle/working/dict"+str(i)+".pkl"
    with open(filename,"rb") as file:
        ref.update(pickle.load(file))
ref

{1: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 2: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 3: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 4: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 5: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 6: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 7: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 8: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 9: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 10: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 11: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 12: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 13: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 14: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 15: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 16: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 17: <MultiTrace: 2 chains, 1000 iterations, 5 variables>,
 18: <

In [12]:
def calculate_result(row):
    site=int(row['SiteId'])
    a=ref[site]['alpha'].mean()
    b0=ref[site]['beta'][0].mean()
    b1=ref[site]['beta'][1].mean()
    b2=ref[site]['beta'][2].mean()
    result = a + b0 * row['AQI'] + b1 * row['PM10']+  b2 * row['PM2.5_AVG']
    return result
df['result'] = df.apply(calculate_result, axis=1)

In [13]:
df.head()

,AQI,PM10,PM2.5_AVG,PM2.5,SiteId,result
42427,1.288729,2.298094,1.102887,25.0,70,29.838451
42428,0.719402,0.422990,0.853072,25.0,71,23.785210
42429,1.810612,3.067368,1.102887,29.0,84,33.984379
42430,0.244963,-0.298204,0.478350,15.0,63,18.029810
42431,0.719402,0.807627,0.853072,23.0,65,22.705133


In [19]:
mse = mean_squared_error(df['PM2.5'].values, df['result'].values)
print("training mse:",mse)

training mse: 20.62958828107301


In [16]:
truth['result'] = df.apply(calculate_result, axis=1)
truth.head()

,SiteName,County,AQI,Pollutant,Status,SO2,CO,CO_8hr,O3,O3_8hr,...,NOx,NO,WindSpeed,WindDirec,PublishTime,SO2_AVG,Longitude,Latitude,SiteId,result
42427,永和,新北市,89.0,懸浮微粒,普通,0.0,0.30,0.3,40.7,43.7,...,10.8,0.2,1.1,95,2023-04-24 00:00:00,0.0,121.516306,25.017000,70,29.838451
42428,復興,高雄市,77.0,細懸浮微粒,普通,1.6,0.38,0.3,24.7,43.7,...,18.5,0.0,0.7,284,2023-04-24 00:00:00,1.0,120.312017,22.608711,71,23.785210
42429,富貴角,新北市,100.0,懸浮微粒,普通,0.2,0.14,0.1,61.5,64.1,...,1.2,0.0,4.8,99,2023-04-24 00:00:00,0.0,121.536569,25.296817,84,33.984379
42430,花蓮,花蓮縣,67.0,細懸浮微粒,普通,0.7,0.27,0.2,44.7,49.2,...,4.5,0.0,0.3,311,2023-04-24 00:00:00,0.0,121.599769,23.971306,63,18.029810
42431,宜蘭,宜蘭縣,77.0,懸浮微粒,普通,1.0,0.20,0.2,42.8,50.7,...,3.4,0.5,0.3,221,2023-04-24 00:00:00,0.0,121.746394,24.747917,65,22.705133


In [18]:
truth=truth.dropna()
mse = mean_squared_error(truth['PM2.5'].values, truth['result'].values)
print("total mse:",mse)

total mse: 25.3495746688444
